In [1]:
import os
import numpy as np
from scipy import spatial as sspat
import pandas as pd
import matplotlib.pyplot as plt

/Users/ivasenkov15/miniconda3/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [32]:
EPS = 1e-12


def calculate_relative_L1_error(nodes1 :np.ndarray, values1 :np.ndarray, nodes2 :np.ndarray, values2 :np.ndarray):
    """
    
    1. Map mesh nodes1 -> mesh nodes2 (Nearest Neighbor)
    2. Denominator = MAX(|Vc|, |Vf|)
    3. Metric = Sum( |Diff| / Denom ) / N_nodes1
    """
    if nodes1 is None or nodes2 is None:
        return 0.0

    n = values1.shape[0] * values1.shape[1]
    
    # Map nodes1 to nodes2
    tree = sspat.cKDTree(nodes2)
    dists, idx = tree.query(nodes1, k=1)
    values2_mapped = values2[idx]
    print(max(dists))
    
    # Denominator: Max of absolute values
    denominator = np.maximum(np.abs(values1), np.abs(values2_mapped))
    
    # Numerator: Absolute Difference
    diff = np.abs(values1 - values2_mapped)
    
    # Filter valid comparisons
    mask = denominator > EPS
    
    if np.sum(mask) == 0:
        return 0.0
        
    # Sum of Relative Differences
    valid_terms = diff[mask] / denominator[mask]
    sum_diff = np.sum(valid_terms)
    
    # Normalize by N
    return sum_diff / n

In [12]:
BASE_FOLDER = 'testcases'


def get_test_folders(base_folder=BASE_FOLDER):
    for _, next_folders, _ in os.walk(base_folder):
        return list(map(lambda x: f'{base_folder}/{x}', filter(lambda x: x.startswith('test'), next_folders)))
    return None


def get_test_filename(folder, label):
    for _, _, filenames in os.walk(folder):
        for filename in filenames:
            if label in filename:
                return f'{folder}/{filename}'
    return None


def read_data(filename, coord_cols, vals_cols):
    ds = pd.read_csv(filename)
    return np.asarray(ds[coord_cols]), np.asarray(ds[vals_cols])


def get_relative_error(test_folder, coord_cols, vals_cols, matlab_label, ansys_label):
    matlab_filename = get_test_filename(test_folder, matlab_label)
    ansys_filename = get_test_filename(test_folder, ansys_label)
    if matlab_filename is None or ansys_filename is None:
        return None

    matlab_nodes, matlab_values = read_data(matlab_filename, coord_cols, vals_cols)
    ansys_nodes, ansys_values = read_data(ansys_filename, coord_cols, vals_cols)
    return calculate_relative_L1_error(matlab_nodes, matlab_values, ansys_nodes, ansys_values)

In [23]:
DEFAULT_COORD_COLS = ['X', 'Y', 'Z']
DEFAULT_DISPLACEMENT_VALS_COLS = ['u']
DEFAULT_STRESS_COLS = ['sigma_x', 'sigma_y', 'sigma_z']
TESTCASES = [
    ('Displacement: mesh_size = 0.01', 'MATLAB_0.01', 'ANSYS_0.01', DEFAULT_COORD_COLS, DEFAULT_DISPLACEMENT_VALS_COLS),
    ('Stress: mesh_size = 0.01', 'MATLAB_0.01', 'ANSYS_0.01', DEFAULT_COORD_COLS, DEFAULT_STRESS_COLS),
    ('Displacement: mesh_size = 0.007', 'MATLAB_0.007', 'ANSYS_0.007', DEFAULT_COORD_COLS, DEFAULT_DISPLACEMENT_VALS_COLS),
    ('Stress: mesh_size = 0.007', 'MATLAB_0.007', 'ANSYS_0.007', DEFAULT_COORD_COLS, DEFAULT_STRESS_COLS),
]

In [33]:
MAX_ERR = 0.05

for test_folder in get_test_folders():
    print(f'{test_folder}:')
    for test_name, matlab_label, ansys_label, coord_cols, vals_cols in TESTCASES:
        rel_err = get_relative_error(test_folder, coord_cols, vals_cols, matlab_label, ansys_label)
        prefix = f'    {test_name}'
        if rel_err is not None:
            print(f'{prefix}: {rel_err:.6f}')
            # assert rel_err < MAX_ERR
        else:
            print(f'{prefix}: None')
    print()

testcases/test_naca0012:
0.007418729726783216
    Displacement: mesh_size = 0.01: 0.924558
0.007418729726783216
    Stress: mesh_size = 0.01: 0.971859
0.005463138005307459
    Displacement: mesh_size = 0.007: 0.931286
0.005463138005307459
    Stress: mesh_size = 0.007: 0.974161

testcases/test_naca0015:
    Displacement: mesh_size = 0.01: None
    Stress: mesh_size = 0.01: None
    Displacement: mesh_size = 0.007: None
    Stress: mesh_size = 0.007: None

testcases/test_naca1234:
    Displacement: mesh_size = 0.01: None
    Stress: mesh_size = 0.01: None
    Displacement: mesh_size = 0.007: None
    Stress: mesh_size = 0.007: None

testcases/test_naca2412:
    Displacement: mesh_size = 0.01: None
    Stress: mesh_size = 0.01: None
    Displacement: mesh_size = 0.007: None
    Stress: mesh_size = 0.007: None

